In [1]:
import importlib
import math
import torch
import os
import sys

import numpy as np
import pandas as pd
from datasets import Dataset

sys.path.append("../")

import src

importlib.reload(src)

from src.data_prep_utils import (  # noqa: E402
    conala_to_time_batches,
    load_time_sorted_conala,
    prep_for_hf,
)

importlib.reload(src.data_prep_utils)

from src.model import load_model_tok

importlib.reload(src.model)


from src.training import train_evaluate, dd_analysis
importlib.reload(src.training)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'src.training' from '/usr/net/zinovyee.hub/IRTG/MLSC/MLSC_DD/notebooks/../src/training.py'>

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
MODEL = "CodeT5"
TRAIN_N = 900
BATCH_SIZE = 35
DECODER_LENGTH = 20
ENCODER_LENGTH = 15

TRAIN_ARGS = {
    "TRAIN_N": TRAIN_N,
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "MODEL": MODEL,
    "SEQ_TRAINER_ARGS": {
        "output_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/results",
        "num_train_epochs": 2,
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 5e-4,
        "warmup_steps": 100,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/logs",
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "epoch",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
    },
}

In [4]:
df = load_time_sorted_conala("../data/raw/conala")  # noqa: PD901
df = conala_to_time_batches(df, TRAIN_N, BATCH_SIZE)  # noqa: PD901

Unique questions: 2074
Sort Question IDs
Create t=0 training sample


In [7]:
model, tokenizer = load_model_tok(MODEL, freeze=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

In [8]:
loss_df = dd_analysis(df, tokenizer=tokenizer,model=model, training_args=TRAIN_ARGS)

Map:   0%|          | 0/1427 [00:00<?, ? examples/s]

Validation for time step 0


Map:   0%|          | 0/42 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.759800,3.793849,0.377100,0.143700,0.360900,0.359500,11.976200,0.049300,0.969900,0.970300,392,404
2,2.802600,3.799844,0.397400,0.149500,0.368000,0.365500,12.333300,0.061600,0.987500,0.987600,399,404


TypeError: eval() missing 1 required positional argument: 'tokenizer'

Map:   0%|          | 0/1427 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [16]:
train_evaluate(train_data, validation_data, TRAIN_ARGS, model, tokenizer)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.064        0.181        0.028        0.154           0.154   

   eval_bleu  eval_gen_len  
0       0.02        10.571  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.762300,3.803319,0.329800,0.114400,0.312000,0.312500,11.500000,0.044500,0.901400,0.905900,366,404
2,2.795100,3.768154,0.353900,0.114800,0.329500,0.328400,11.595200,0.041900,0.946600,0.948000,383,404


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      3.768        0.354        0.115         0.33           0.328   

   eval_bleu  eval_gen_len  
0      0.042        11.595  


ImportError: cannot import name 'dd_analysis' from 'src.training' (/usr/net/zinovyee.hub/IRTG/MLSC/MLSC_DD/notebooks/../src/training.py)